Capstone by Joel Harlander
Code is all in python 3.5

# The Goal:
Using word frequencies extracted purely fictional texts, are we able to pull out and predict if a real world event is occuring?

The dataset is large, extra effort was needed to even download the data. 

http://storage.googleapis.com/books/ngrams/books/datasetsv2.html 

Data is in a TSV format: 

ngram TAB year TAB match_count TAB page_count TAB volume_count NEWLINE

A good description of an n-gram can be found here: http://text-analytics101.rxnlp.com/2014/11/what-are-n-grams.html

The first step is to collect the data from the ngram website.
Used requests 
The download.py had to be run several times as we were not able to download it quick enough to get in in one (or several) sitting(s)

Code here:

In [ ]:
# import requests
# from bs4 import BeautifulSoup as bs
# import urllib
# import os


# _URL = 'http://storage.googleapis.com/books/ngrams/books/datasetsv2.html'

# # Make a list of everything i want
# r = requests.get(_URL)
# soup = bs(r.text, "lxml")
# names = []
# for i, link in enumerate(soup.findAll('a')):
#     print(i)
#     # _FULLURL = _URL + link.get('href')
#     _FULLURL = link.get('href')
#     if _FULLURL.endswith('.gz') and '-eng-' in _FULLURL and '20120701' in _FULLURL:
#         print(''.join([
#             str(i),
#             ' - Adding: ',
#             soup.select('a')[i].attrs['href']
#             ]))

#         names.append(soup.select('a')[i].attrs['href'])

# # names_urls = zip(names, urls)

# for i, name in enumerate(names):
#     # number im upto as there is no access to a connection to be able to download all this in one session
#     if i < 486:
#         pass
#     else:
#         try:
#             os.remove("/media/sf_VMData/" + name.split('/')[-1])
#         except:
#             print(''.join([
#                 '\nNow downloading ',
#                 str(i),
#                 ' of ',
#                 str(len(names)),
#                 '\nFilename: ',
#                 str(name)
#                 ]))

#             res = urllib.request.urlopen(name)
#             DL_file = open("/media/sf_VMData/" + name.split('/')[-1], 'wb')
#             DL_file.write(res.read())
#             DL_file.close()

A few days later we had all the fictional data.

All files contained a single tsv file compressed in a .tar.gz format.

Here is where I ran into my first limitations:

- I didn't actually have enough space to extract all the files to disk
- A lot of the files were too big to read into python as they exceeded 12gb (the amount of memory assigned to my VM)
- Data was split by alphabet rather than time, this would mean if I wanted to get a slice of time I'd need every single file

PostgreSQL has some ability to compress data, with this in mind I decided to explore this path.
As reading data into memory was not an option I used python and the gzip library to decompress on the fly and read the contained zip file line-by-line and insert those values into the database.

Code here:


In [ ]:
# import urllib
# import os
# import gzip

# from sqlalchemy import create_engine
# import psycopg2


# import numpy as np
# import pandas as pd


# def con_sql(db='postgres', usr='j', loc='localhost', pswd='ilovedatascience'):
#     conn = psycopg2.connect(dbname=db, user=usr, host=loc, password=pswd)
#     return conn


# conn = con_sql()
# c = conn.cursor()

# def sql_qu(st, c=conn):
#     df = pd.read_sql(st, con=conn)
#     return df

# # create = '''
# #     CREATE TABLE "ngrams" (
# #     "n" INTEGER,
# #     "ngram" TEXT,
# #     "year" INTEGER,
# #     "match_count" INTEGER,
# #     "volume_count" INTEGER)
# #     ;
# # '''
# # c.execute("DROP TABLE ngrams;")
# # conn.commit()
# # c.execute(create)
# # conn.commit()
# job_n = int(input('How many files to process:\n>>'))
# for item in os.listdir('/media/sf_VMData/'):
#     if 'googlebooks-eng-fiction-all-5gram' in item and item.endswith('.gz'):
#         print('Extracting: ', item)
#         with gzip.open('/media/sf_VMData/'+item, 'rt') as f:
#             file_content = f.read().splitlines()
#             for line in file_content:
#                 ngram = str(line.split('\t')[0]).replace("\'","\'\'")
#                 year = str(line.split('\t')[1])
#                 match_count = str(line.split('\t')[2])
#                 volume_count = str(line.split('\t')[3])

#                 strsql = 'INSERT INTO ngrams VALUES (' + item[28] + ',' + "'" + ngram +"'" + ','+year+','+ match_count+','+ volume_count+')'
#                 c.execute(strsql)
#                 conn.commit()
#         os.rename('/media/sf_VMData/'+item, '/media/sf_VMData/done/'+item)
#     elif 'skip' not in item and 'done' not in item:
#         print('yeah, nah: ', item)
#         os.rename('/media/sf_VMData/'+item, '/media/sf_VMData/skip/'+item)
#     else:
#         pass
#     job_n -= 1
#     if job_n <= 0:
#         break

As you may of noticed it asks how many files i would like to use and stores it in a variable called job_n:

job_n = int(input('How many files to process:\n>>'))


Also as a side note:
During the debugging process I had included in the code c.execute("DROP TABLE ngrams;") in order to start from a clean slate each time.

Forgetting to remove this when reading the data into SQL in daily batches (dropping the table of data read in from the previous day) taught me an important lesson to check and double check your code is doing what you want before running it with an absent mind.

This process was also very slow, and in the end I only used the 1-gram data due to time and processing contraints.


## Data input challanges:

Hadoop/spark clusted lwere setup, however due to the timeline of the project and the learning curve of using the technolgies there was not suffiecnt time to utilize these
Issues with code optimisation, non optimal code often made the difference between seconds and hours.


Link to the next notebook: [EDA and Analysis](2. EDA and Analysis.ipynb)